### Generating Hume-like gibberish

This document is a demo for my LSTM implementation.

In [1]:
import string
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.optim import Adam

from training import train
from data_handling import DataLoader
from model import LSTM

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
all_letters = string.ascii_letters + " 0123456789!?,.:;'()[]-$"

In [4]:
data_loader = DataLoader("hume.txt", 
                         all_letters=all_letters, 
                         seq_length=500, 
                         device=device, 
                         verbose=True)

precomputing data tensors...
20.00%
39.99%
59.99%
79.99%
99.98%
100%: done


In [5]:
rnn = LSTM(n_input=data_loader.n_letters, 
           n_hidden=512, 
           n_output=data_loader.n_letters, 
           n_layers=3,
           dropout=0.1, 
           grad_clipping=5).to(device)

In [ ]:
training_params = {
    'number of iterations':3000,
    'batch size':128,
    'subsequence length':100,
    'learning rate':0.0005
}

output_params = {
    'plot every':50,
    'print every':200
}

iters, losses = train(rnn, data_loader, training_params, output_params)

  1m 50s (200 6.67%) loss: 0.00013
  3m 39s (400 13.33%) loss: 0.00010
  5m 28s (600 20.00%) loss: 0.00008
  7m 20s (800 26.67%) loss: 0.00008
  9m 11s (1000 33.33%) loss: 0.00007


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(losses)
plt.show()

### Testing output

Here, we can finally sample from the learned distribution.

In [ ]:
max_sample_length = 2000

def sample(starting_with="", temperature=1):
    verse = starting_with
    with torch.no_grad():
        rnn.eval()
        letter = "$" + starting_with
        state = rnn.init_state(data_loader.seq2tensor(letter, truncate=False).unsqueeze(1).to(device))
        for i in range(max_sample_length):
            input_tensor = data_loader.seq2tensor(letter, truncate=False).unsqueeze(1).to(device)
            output, state = rnn.sample(input_tensor, state, temperature)
            probs = np.exp(output[-1].squeeze().cpu().numpy())
            letter = np.random.choice(list(data_loader.all_letters), p=probs)
            if letter == "$":
                return verse
            verse += letter
    return verse

for i in range(10):
    print(sample("", temperature=0.9) + '\n')